In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("Case_Study_Spark").getOrCreate()

In [ ]:
loanDf=spark.read.csv("loan.csv",header=True,inferSchema=True)
loanDf.show()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

#number of loans in each category

In [ ]:

loanDf.groupBy("Loan Category").count().show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



#number of people who have taken more than 1 lack loan

In [ ]:
from pyspark.sql.functions import col,regexp_replace
loanDf = loanDf.withColumn("LoanAmountClean",regexp_replace("Loan Amount",",","").cast("int"))
loanDf.filter(col("LoanAmountClean")>100000).count()


450

#number of people with income greater than 60000 rupees

In [ ]:
loanDf.filter(col("Income")>60000).count()

198

#number of people with 2 or more returned cheques and income less than 50000

In [ ]:
loanDf.filter((col(" Returned Cheque")>=2) & (col("Income")<50000)).count()

137

#number of people with 2 or more returned cheques and are single

In [ ]:
loanDf.filter((col(" Returned Cheque")>=2) & (col("Marital Status")<"SINGLE")).count()

283

#number of people with expenditure over 50000 a month

In [ ]:
loanDf.filter(col("Expenditure")>50000).count()

6

#number of members who are elgible for credit card

In [ ]:
loanDf.filter((col("Income") > 50000) & (col("Age") >= 20) & (col("Age") <= 60) & (col("Overdue") <= 6) & (col(" Returned Cheque") <= 6) & (col(" Dishonour of Bill") <= 6)).count()

125

In [ ]:
creditDf=spark.read.csv("credit card.csv",header=True,inferSchema=True)
creditDf.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|F

#credit card users in Spain

In [ ]:
creditDf.filter(col("Geography")=="Spain").select("CustomerId","Surname","Age","Gender").show()

+----------+---------+---+------+
|CustomerId|  Surname|Age|Gender|
+----------+---------+---+------+
|  15647311|     Hill| 41|Female|
|  15737888| Mitchell| 43|Female|
|  15574012|      Chu| 44|  Male|
|  15737173|  Andrews| 24|  Male|
|  15600882|    Scott| 35|Female|
|  15788218|Henderson| 24|Female|
|  15661507|  Muldrow| 45|  Male|
|  15597945| Dellucci| 32|Female|
|  15699309|Gerasimov| 38|Female|
|  15589475|  Azikiwe| 39|Female|
|  15659428|  Maggard| 42|Female|
|  15732963| Clements| 29|Female|
|  15788448|   Watson| 31|  Male|
|  15729599|  Lorenzo| 33|  Male|
|  15619360|    Hsiao| 40|  Male|
|  15684171|  Bianchi| 61|Female|
|  15623944|    T'ien| 66|Female|
|  15702014|  Jeffrey| 33|  Male|
|  15751208|  Pirozzi| 56|  Male|
|  15812518|  Palermo| 37|Female|
+----------+---------+---+------+
only showing top 20 rows



#number of members who are elgible and active in the bank

In [ ]:
creditDf.filter((col("IsActiveMember")==1)&(col("Exited")==1)).count()

735

In [ ]:
txnDf=spark.read.csv("txn.csv",header=True,inferSchema=True)
txnDf.show()
txnDf.count()
txnDf.printSchema()


+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            NULL|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            NULL|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            NULL|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            NULL|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  7500000.0|

#Maximum withdrawal amount in transactions

In [ ]:
from pyspark.sql.functions import max
txnDf.select(max(" WITHDRAWAL AMT ").alias("Max withdrawal")).show()

+--------------+
|Max withdrawal|
+--------------+
| 4.594475464E8|
+--------------+



#MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT in txn.csv

In [ ]:
from pyspark.sql.functions import min
txnDf.select(min(" WITHDRAWAL AMT ").alias("Min withdrawal")).show()

+--------------+
|Min withdrawal|
+--------------+
|          0.01|
+--------------+



#MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT

In [ ]:
txnDf.select(max(" DEPOSIT AMT ").alias("Max deposit")).show()

+-----------+
|Max deposit|
+-----------+
|    5.448E8|
+-----------+



#MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT

In [ ]:
txnDf.select(min(" DEPOSIT AMT ").alias("Min deposit")).show()

+-----------+
|Min deposit|
+-----------+
|       0.01|
+-----------+



In [ ]:
txnDf.filter(col("BALANCE AMT") <0).show()
txnDf.filter(col("BALANCE AMT") <0).count()

+-------------+--------------------+----------+----------------+-------------+--------------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |   BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+--------------+
|409000425051'|TRF TO Rajendra K...| 26-Oct-18|           1.5E7|         NULL|-1.454079965E7|
|409000425051'|  TRF TO  Myur Joshi| 31-Oct-18|          3.54E8|         NULL|-3.535407997E8|
|409000425051'|TRF TO  Indiafore...| 31-Oct-18|        900000.0|         NULL|-3.544407997E8|
|409000425051'|409000425051:Int....| 31-Oct-18|         81569.0|         NULL|-3.545223687E8|
|409000425051'|TRF FROM  Indiafo...| 27-Nov-18|            NULL|      81568.0|-3.544408007E8|
|409000425051'|TRF FROM  Indiafo...| 27-Nov-18|            NULL|      10000.0|-3.544308007E8|
|409000425051'|409000425051:Int....| 30-Nov-18|       2447599.0|         NULL|-3.568783997E8|
|409000425051'|TRF FRM  Indiafor...| 17-Dec-18|            N

113276

In [ ]:
from pyspark.sql.functions import sum,count
txnDf.groupBy("Account No").agg(sum("BALANCE AMT").alias("Total Balance")).orderBy(col("Total Balance").desc()).show()

+-------------+--------------------+
|   Account No|       Total Balance|
+-------------+--------------------+
|409000611074'|       1.615533622E9|
|409000493201'|1.0420831829499985E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000438611'|-2.49486577068339...|
|409000493210'|-3.27584952132095...|
|409000438620'|-7.12291867951358...|
|     1196711'|-1.60476498101275E13|
|409000362497'| -5.2860004792808E13|
|     1196428'| -8.1418498130721E13|
+-------------+--------------------+



#Number of transaction on each date

In [ ]:

txnDf.groupBy("VALUE DATE").agg(count("TRANSACTION DETAILS").alias("Total Transaction")).orderBy(col("Total Transaction").desc()).show()

+----------+-----------------+
|VALUE DATE|Total Transaction|
+----------+-----------------+
| 27-Jul-17|              567|
| 13-Aug-18|              463|
|  8-Nov-17|              402|
|  7-Oct-17|              382|
| 10-Jul-18|              374|
| 12-Dec-17|              367|
| 12-Sep-18|              365|
|  9-Aug-18|              360|
| 19-Sep-17|              358|
| 16-Mar-17|              353|
| 10-Sep-18|              344|
| 14-Jul-17|              333|
|  7-Mar-18|              319|
| 11-Oct-18|              303|
| 22-Aug-17|              301|
|  9-Jan-18|              299|
|  9-Oct-18|              297|
| 20-Apr-18|              296|
|  9-Jul-18|              292|
|  7-Apr-18|              291|
+----------+-----------------+
only showing top 20 rows



In [ ]:
txnDf.filter(col(" WITHDRAWAL AMT ")>100000).show()

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        133900.0|         NULL|  8366100.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        195800.0|         NULL|  8147300.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        143800.0|         NULL|  7781600.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        331650.0|         NULL|  7449950.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        129000.0|         NULL|  7320950.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        230013.0|         NULL|  7090937.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        367900.0|         NULL|  6723037.0|
|409000611074'|INDO GIBL Indiafo...| 16-Aug-17|        108000.0|         NULL|  6615037.0|